In [44]:
import numpy as np
import pandas as pd
import re

In [26]:
data = pd.read_csv('./sentiment_analysis_dataset.csv',encoding = 'utf-8',error_bad_lines=False)

b'Skipping line 5944: expected 2 fields, saw 8\n'


In [34]:
data.columns = ['class','text']
data.head()

,class,text
0,neg,a sentimental mess that never rings true .
1,neg,"while the performances are often engaging , th..."
2,neg,"interesting , but not compelling ."
3,neg,on a cutting room floor somewhere lies . . . f...
4,neg,while the ensemble player who gained notice in...


In [41]:
def text_to_wordlist(text):
   
    # 用正则表达式取出符合规范的部分
    review_text = re.sub("[^a-zA-Z]"," ", text)
    # 小写化所有的词，并转成词list
    words = review_text.lower().split()
    # 返回words
    return words

In [45]:
# 预处理数据
label = data['class']
train_data = []
for i in range(len(data['text'])):
    train_data.append(' '.join(text_to_wordlist(data['text'][i])))


In [46]:
train_data[0]

'a sentimental mess that never rings true'

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF

tfidf = TFIDF(min_df=2, # 最小支持度为2
           max_features=None,
           strip_accents='unicode',
           analyzer='word',
           token_pattern=r'\w{1,}',
           ngram_range=(1, 3),  # 二元文法模型
           use_idf=1,
           smooth_idf=1,
           sublinear_tf=1,
           stop_words = 'english') # 去掉英文停用词

# 合并训练和测试集以便进行TFIDF向量化操作
data_all = train_data
len_train = len(train_data)

tfidf.fit(data_all)
data_all = tfidf.transform(data_all)
train_x = data_all[:]
print ('TF-IDF处理结束.')

TF-IDF处理结束.


In [55]:
from sklearn.naive_bayes import MultinomialNB as MNB

model_NB = MNB()
model_NB.fit(train_x, label)
MNB(alpha=1.0, class_prior=None, fit_prior=True)

#from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score


print ("多项式贝叶斯分类器10折交叉验证得分: ", np.mean(cross_val_score(model_NB, train_x, label, cv=10, scoring='roc_auc')))

多项式贝叶斯分类器10折交叉验证得分:  0.8572148069013327
